# [Module] CatBoost 로 훈련 및 추론

이 노트북은 주로 아래와 같은 작업을 합니다.

---


# 1. 환경 셋업
- 아래는 파이썬 캐키지를 임포트할때에 캐싱된 것을 사용하지 않고, 매번 리로딩 하는 세팅 입니다.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%store -r

In [3]:
import pandas as pd
pd.options.display.max_rows=200
from IPython.display import display as dp

import os, sys
import boto3
import sagemaker

from sagemaker import get_execution_role

role = get_execution_role()



# 2. 훈련 데이터 로딩 및 데이터 정리
- 아래는 데이터 로딩을 위해서 9.6 GB 가 필요합니다. 메모리 할당 에러가 나면 다른 노트북의 커널 세션을 삭제하고 해보세요. 그래도 안되면 메모리가 많은 노트북 인스턴스에서 다시 하시기 바랍니다.

In [4]:
full_local_path = '../../prepare_data/data/full/full-1849039.pkl'

In [5]:
%%time
file_path = full_local_path

df = pd.read_pickle(file_path)
df.head()

CPU times: user 823 ms, sys: 367 ms, total: 1.19 s
Wall time: 1.19 s


,EVENT_LABEL,str_ip,str_app,str_device,str_os,str_channel,EVENT_TIMESTAMP,EVENT_TIMESTAMP_SIMPLE
71816445,0,str_103106,str_3,str_1,str_13,str_371,2017-11-08 00:49:06,2017-11-08 00:00:00
118420665,0,str_83928,str_3,str_1,str_19,str_417,2017-11-08 14:54:59,2017-11-08 14:00:00
2983188,0,str_36150,str_15,str_2,str_49,str_3,2017-11-06 17:27:10,2017-11-06 17:00:00
111499266,0,str_40056,str_26,str_1,str_18,str_477,2017-11-08 13:00:46,2017-11-08 13:00:00
1598682,0,str_39081,str_18,str_1,str_8,str_107,2017-11-06 16:37:52,2017-11-06 16:00:00


In [6]:
sys.path.append('../../prepare_data/src')
from p_utils import drop_column, change_code_to_string

In [7]:
def create_hour_feature(df, src_col, new_col):
    fdf = df.copy()
    fdf.insert(len(fdf.columns),column=new_col, value=fdf[src_col].apply(lambda x: x.hour))        
    
    return fdf
    
df = create_hour_feature(df, src_col='EVENT_TIMESTAMP_SIMPLE', 
                         new_col='EVENT_HOUR')    
df = change_code_to_string(df, col='EVENT_HOUR', new_col='STR_EVENT_HOUR', verbose=False)


In [8]:
df = drop_column(df, col='EVENT_HOUR')
df = df.sort_values(by= ['str_ip', 'EVENT_TIMESTAMP'])
df.head(10)

,EVENT_LABEL,str_ip,str_app,str_device,str_os,str_channel,EVENT_TIMESTAMP,EVENT_TIMESTAMP_SIMPLE,STR_EVENT_HOUR
9889166,0,str_10,str_1,str_1,str_11,str_118,2017-11-07 00:10:14,2017-11-07 00:00:00,str_0
9905499,0,str_10,str_8,str_1,str_11,str_145,2017-11-07 00:10:30,2017-11-07 00:00:00,str_0
14494365,0,str_10,str_12,str_1,str_11,str_265,2017-11-07 01:28:03,2017-11-07 01:00:00,str_1
14939653,0,str_10,str_15,str_1,str_22,str_278,2017-11-07 01:36:08,2017-11-07 01:00:00,str_1
15005081,0,str_10,str_11,str_1,str_22,str_319,2017-11-07 01:37:19,2017-11-07 01:00:00,str_1
15453752,0,str_10,str_8,str_1,str_11,str_145,2017-11-07 01:45:20,2017-11-07 01:00:00,str_1
15746142,0,str_10,str_15,str_1,str_11,str_140,2017-11-07 01:50:53,2017-11-07 01:00:00,str_1
17928835,0,str_10,str_1,str_1,str_9,str_452,2017-11-07 02:32:45,2017-11-07 02:00:00,str_2
39616896,0,str_10,str_3,str_1,str_28,str_280,2017-11-07 09:31:51,2017-11-07 09:00:00,str_9
45593329,0,str_10,str_3,str_1,str_49,str_417,2017-11-07 11:23:38,2017-11-07 11:00:00,str_11


## 데이터의 훈련, 테스트 샘플링

In [9]:
from p_utils import split_data_by_time, save_csv_local
train_df, test_df = split_data_by_time(
                       df=df, 
                       target_col='EVENT_TIMESTAMP', 
                       label_col = 'EVENT_LABEL',
                       total_samples=200000, 
                       split_rate=0.1, 
                       train_end='2017-11-08 23:59', 
                       test_start='2017-11-09 00:00',    
                       verbose = True,
                  )

train sample shape:  (180000, 9)
train min time:  2017-11-06 15:46:38
train max time:  2017-11-08 23:59:00
Train fraud ratio:  0.00264
# of Train frauds:  475

test sample shape:  (20000, 9)
test min time:  2017-11-09 00:00:01
test max time:  2017-11-09 15:59:55
Test fraud ratio:  0.00275
# of test frauds:  55


In [10]:
train_df.head()

,EVENT_LABEL,str_ip,str_app,str_device,str_os,str_channel,EVENT_TIMESTAMP,EVENT_TIMESTAMP_SIMPLE,STR_EVENT_HOUR
69402732,0,str_78173,str_12,str_1,str_15,str_259,2017-11-08 00:08:02,2017-11-08 00:00:00,str_0
88909993,0,str_56188,str_3,str_1,str_19,str_280,2017-11-08 05:58:52,2017-11-08 05:00:00,str_5
52057252,0,str_62464,str_29,str_2,str_9,str_343,2017-11-07 13:30:21,2017-11-07 13:00:00,str_13
36751384,0,str_17310,str_32,str_1,str_19,str_376,2017-11-07 08:32:01,2017-11-07 08:00:00,str_8
28433179,0,str_2944,str_14,str_1,str_13,str_442,2017-11-07 05:41:03,2017-11-07 05:00:00,str_5


In [11]:
test_df.head()

,EVENT_LABEL,str_ip,str_app,str_device,str_os,str_channel,EVENT_TIMESTAMP,EVENT_TIMESTAMP_SIMPLE,STR_EVENT_HOUR
136336890,0,str_171877,str_3,str_1,str_26,str_280,2017-11-09 01:21:38,2017-11-09 01:00:00,str_1
141848554,0,str_102999,str_18,str_1,str_13,str_121,2017-11-09 03:09:12,2017-11-09 03:00:00,str_3
165441931,0,str_35096,str_8,str_1,str_25,str_145,2017-11-09 10:08:50,2017-11-09 10:00:00,str_10
177292494,0,str_17149,str_3,str_2,str_27,str_137,2017-11-09 13:40:25,2017-11-09 13:00:00,str_13
147459346,0,str_83735,str_12,str_1,str_13,str_328,2017-11-09 04:41:19,2017-11-09 04:00:00,str_4


## CatBoost 알고리즘에 사용되지 않는 Timestamp 제거

In [12]:
train_df = drop_column(train_df, col='EVENT_TIMESTAMP')
train_df = drop_column(train_df, col='EVENT_TIMESTAMP_SIMPLE')

In [13]:
train_df.head()

,EVENT_LABEL,str_ip,str_app,str_device,str_os,str_channel,STR_EVENT_HOUR
69402732,0,str_78173,str_12,str_1,str_15,str_259,str_0
88909993,0,str_56188,str_3,str_1,str_19,str_280,str_5
52057252,0,str_62464,str_29,str_2,str_9,str_343,str_13
36751384,0,str_17310,str_32,str_1,str_19,str_376,str_8
28433179,0,str_2944,str_14,str_1,str_13,str_442,str_5


In [14]:
test_df = drop_column(test_df, col='EVENT_TIMESTAMP')
test_df = drop_column(test_df, col='EVENT_TIMESTAMP_SIMPLE')

In [15]:
test_df.head()

,EVENT_LABEL,str_ip,str_app,str_device,str_os,str_channel,STR_EVENT_HOUR
136336890,0,str_171877,str_3,str_1,str_26,str_280,str_1
141848554,0,str_102999,str_18,str_1,str_13,str_121,str_3
165441931,0,str_35096,str_8,str_1,str_25,str_145,str_10
177292494,0,str_17149,str_3,str_2,str_27,str_137,str_13
147459346,0,str_83735,str_12,str_1,str_13,str_328,str_4


# 3. 훈련 및 테스트 데이터 저장

In [16]:
train_file_name = 'train-' + str(train_df.shape[0]) + ".csv"
train_local_path = save_csv_local(raw_df=train_df, preproc_folder='data/train', 
                                  label='EVENT_LABEL', file_name=train_file_name)
print("train_local_path: ", train_local_path)

test_file_name = 'test-' + str(test_df.shape[0]) + ".csv"
test_local_path = save_csv_local(raw_df=test_df, preproc_folder='data/test', 
                                  label='EVENT_LABEL', file_name=test_file_name)
print("test_local_path: ", test_local_path)

data/train/train-180000.csv is saved
train_local_path:  data/train/train-180000.csv
data/test/test-20000.csv is saved
test_local_path:  data/test/test-20000.csv


In [17]:
import sagemaker

bucket = sagemaker.Session().default_bucket()
# 프로젝트 변수
project_prefix = 'adtalking_fraud_phase0'

# S3에 저장되는 데이터의 기본 폴더 위치
s3_train_data_uri = f"s3://{bucket}/{project_prefix}/train"
s3_test_data_uri = f"s3://{bucket}/{project_prefix}/test"

In [18]:
s3_train_data_uri = sagemaker.s3.S3Uploader.upload(
    local_path=train_local_path, 
    desired_s3_uri=s3_train_data_uri,    
)
print("s3_train_data_uri: \n", s3_train_data_uri)

s3_test_data_uri = sagemaker.s3.S3Uploader.upload(
    local_path=test_local_path, 
    desired_s3_uri=s3_test_data_uri,    
)
print("s3_test_data_uri: \n", s3_test_data_uri)

s3_train_data_uri: 
 s3://sagemaker-us-east-1-057716757052/adtalking_fraud_phase0/train/train-180000.csv
s3_test_data_uri: 
 s3://sagemaker-us-east-1-057716757052/adtalking_fraud_phase0/test/test-20000.csv


# 4. 로컬모드로 모델 훈련

In [21]:
from sagemaker.sklearn.estimator import SKLearn

FRAMEWORK_VERSION = "0.23-1"

local_sklearn = SKLearn(
    base_job_name = "catboost-training",
    entry_point="catboost_training.py",    
    source_dir='source', 
    framework_version=FRAMEWORK_VERSION,
    instance_type="local",    
    role=role,
    hyperparameters={'cat-features': 'str_ip str_app str_device str_os str_channel STR_EVENT_HOUR',
                     'target': 'EVENT_LABEL',
                    'train-file': 'train-180000.csv'},
)

In [22]:
local_sklearn.fit({'train':s3_train_data_uri}, wait=True)

Creating ascg5a06vn-algo-1-2vtfq ... 
Creating ascg5a06vn-algo-1-2vtfq ... done
Attaching to ascg5a06vn-algo-1-2vtfq
ascg5a06vn-algo-1-2vtfq | 2021-11-10 01:40:40,470 sagemaker-containers INFO     Imported framework sagemaker_sklearn_container.training
ascg5a06vn-algo-1-2vtfq | 2021-11-10 01:40:40,472 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
ascg5a06vn-algo-1-2vtfq | 2021-11-10 01:40:40,481 sagemaker_sklearn_container.training INFO     Invoking user training script.
ascg5a06vn-algo-1-2vtfq | 2021-11-10 01:40:40,607 sagemaker-training-toolkit INFO     Installing dependencies from requirements.txt:
ascg5a06vn-algo-1-2vtfq | /miniconda3/bin/python -m pip install -r requirements.txt
ascg5a06vn-algo-1-2vtfq | Collecting catboost==0.26
ascg5a06vn-algo-1-2vtfq |   Downloading catboost-0.26-cp37-none-manylinux1_x86_64.whl (69.2 MB)
     |████████████████████████████████| 69.2 MB 18.4 MB/s eta 0:00:01
ascg5a06vn-algo-1-2vtfq | Requirement already satisf

# 5. 호스트 모드로 훈련

In [23]:
from sagemaker.sklearn.estimator import SKLearn

FRAMEWORK_VERSION = "0.23-1"
sess = sagemaker.session.Session()

sklearn = SKLearn(
    base_job_name = "catboost-training",
    entry_point="catboost_training.py",    
    source_dir='source', 
    framework_version=FRAMEWORK_VERSION,
    instance_type="ml.c5.2xlarge",    
    role=role,
    sagemaker_session=sess,    
    hyperparameters={'cat-features': 'str_ip str_app str_device str_os str_channel STR_EVENT_HOUR',
                     'target': 'EVENT_LABEL',
                    'train-file': 'train-180000.csv'},
)

sklearn.fit({'train':s3_train_data_uri}, wait=False)

In [24]:
sklearn.logs()

2021-11-10 01:40:56 Starting - Starting the training job...
2021-11-10 01:41:24 Starting - Launching requested ML instancesProfilerReport-1636508456: InProgress
......
2021-11-10 01:42:25 Starting - Preparing the instances for training.........
2021-11-10 01:43:52 Downloading - Downloading input data
2021-11-10 01:43:52 Training - Downloading the training image..2021-11-10 01:44:08,049 sagemaker-containers INFO     Imported framework sagemaker_sklearn_container.training
2021-11-10 01:44:08,051 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2021-11-10 01:44:08,060 sagemaker_sklearn_container.training INFO     Invoking user training script.

2021-11-10 01:44:25 Training - Training image download completed. Training in progress.2021-11-10 01:45:08,436 sagemaker-training-toolkit INFO     Installing dependencies from requirements.txt:
/miniconda3/bin/python -m pip install -r requirements.txt
You should consider upgrading via the '/miniconda3/bin/python -m

# 6. 추론 함수 테스트

## 모델 아티펙트 다운로드 및 압축해제
- 모델 아티펙트를 다운로드 합니다.
- 다운로드 받은 모델 아티펙트의 압축을 해제하고 모델 가중치인 model.pth 파일을 얻습니다.

In [25]:
artifact_path = local_sklearn.model_data
print("artifact_path: ", artifact_path)

artifact_path:  s3://sagemaker-us-east-1-057716757052/catboost-training-2021-11-10-01-40-37-755/model.tar.gz


In [26]:
import os

model_data_dir = 'model/script'
os.makedirs(model_data_dir, exist_ok=True)


In [27]:
%%sh -s {artifact_path} {model_data_dir}

echo $1
echo $2

aws s3 cp $1 $2
cd $2
tar -xvf model.tar.gz  

s3://sagemaker-us-east-1-057716757052/catboost-training-2021-11-10-01-40-37-755/model.tar.gz
model/script
download: s3://sagemaker-us-east-1-057716757052/catboost-training-2021-11-10-01-40-37-755/model.tar.gz to model/script/model.tar.gz
model.joblib


## 모델 로딩 함수 테스트

In [28]:
import sys, os
from source.catboost_training  import model_fn , input_fn, predict_fn

joblib version:  1.0.1


In [29]:
model = model_fn(model_data_dir)

############### Model_FN() ###################
### Model loaded 


## Input 함수 테스트

## 테스트 데이터 로딩

In [30]:
from src.sm_utils import get_payload

In [31]:
sample_df = test_df.head(5)

In [33]:
payload = get_payload(sample_df, label_col = 'EVENT_LABEL', verbose=False)
payload



'str_171877,str_3,str_1,str_26,str_280,str_1\nstr_102999,str_18,str_1,str_13,str_121,str_3\nstr_35096,str_8,str_1,str_25,str_145,str_10\nstr_17149,str_3,str_2,str_27,str_137,str_13\nstr_83735,str_12,str_1,str_13,str_328,str_4\n'

In [34]:
result_input_fn = input_fn(payload, content_type='text/csv')
print("result_input_fn: ", result_input_fn.shape)



#### input_fn starting ######
data type: <class 'str'>
input is 
 str_171877,str_3,str_1,str_26,str_280,str_1
str_102999,str_18,str_1,str_13,str_121,str_3
str_35096,str_8,str_1,str_25,str_145,str_10
str_17149,str_3,str_2,str_27,str_137,str_13
str_83735,str_12,str_1,str_13,str_328,str_4

After tansformation, data type: <class 'numpy.ndarray'>
input is 
 [['str_171877' 'str_3' 'str_1' 'str_26' 'str_280' 'str_1']
 ['str_102999' 'str_18' 'str_1' 'str_13' 'str_121' 'str_3']
 ['str_35096' 'str_8' 'str_1' 'str_25' 'str_145' 'str_10']
 ['str_17149' 'str_3' 'str_2' 'str_27' 'str_137' 'str_13']
 ['str_83735' 'str_12' 'str_1' 'str_13' 'str_328' 'str_4']]
result_input_fn:  (5, 6)


## Predict 함수 테스트

In [35]:
output = predict_fn(result_input_fn, model)
#output = predict_fn(images, model)
output

############### predict_FN() ###################
type:  <class 'numpy.ndarray'>
input: 
 [['str_171877' 'str_3' 'str_1' 'str_26' 'str_280' 'str_1']
 ['str_102999' 'str_18' 'str_1' 'str_13' 'str_121' 'str_3']
 ['str_35096' 'str_8' 'str_1' 'str_25' 'str_145' 'str_10']
 ['str_17149' 'str_3' 'str_2' 'str_27' 'str_137' 'str_13']
 ['str_83735' 'str_12' 'str_1' 'str_13' 'str_328' 'str_4']]


array([[0.81956982, 0.18043018],
       [0.81956982, 0.18043018],
       [0.81956982, 0.18043018],
       [0.81869713, 0.18130287],
       [0.81956982, 0.18043018]])

# 7. 로컬 엔드포인트에서 추론 

In [36]:
from datetime import datetime
suffix = datetime.now().microsecond

instance_type='local'
local_endpoint_name = 'local-endpoint-' + str(suffix)
print("local_endpoint_name: ", local_endpoint_name)


local_endpoint_name:  local-endpoint-68739


In [37]:
# 로컬 노트북 인스턴스에서 사용할 로컬 세션
local_session = sagemaker.local.LocalSession()
local_session.config = {'local' : {'local_code':True}}

In [38]:
dummy_endpont = local_sklearn.deploy(
    initial_instance_count=1, 
#     instance_type="ml.c5.xlarge"
    instance_type="local",    
    endpoint_name = local_endpoint_name    
)

Attaching to zjbmhkpbn8-algo-1-559cq
zjbmhkpbn8-algo-1-559cq | 2021-11-10 01:48:07,377 INFO - sagemaker-containers - No GPUs detected (normal if no gpus installed)
zjbmhkpbn8-algo-1-559cq | 2021-11-10 01:48:07,381 INFO - sagemaker-containers - No GPUs detected (normal if no gpus installed)
zjbmhkpbn8-algo-1-559cq | 2021-11-10 01:48:07,382 INFO - sagemaker-containers - nginx config: 
zjbmhkpbn8-algo-1-559cq | worker_processes auto;
zjbmhkpbn8-algo-1-559cq | daemon off;
zjbmhkpbn8-algo-1-559cq | pid /tmp/nginx.pid;
zjbmhkpbn8-algo-1-559cq | error_log  /dev/stderr;
zjbmhkpbn8-algo-1-559cq | 
zjbmhkpbn8-algo-1-559cq | worker_rlimit_nofile 4096;
zjbmhkpbn8-algo-1-559cq | 
zjbmhkpbn8-algo-1-559cq | events {
zjbmhkpbn8-algo-1-559cq |   worker_connections 2048;
zjbmhkpbn8-algo-1-559cq | }
zjbmhkpbn8-algo-1-559cq | 
zjbmhkpbn8-algo-1-559cq | http {
zjbmhkpbn8-algo-1-559cq |   include /etc/nginx/mime.types;
zjbmhkpbn8-algo-1-559cq |   default_type application/octet-stream;
zjbmhkpbn8-algo-1-559c

## 프리딕터 생성

In [39]:
from sagemaker.deserializers import CSVDeserializer
csv_deserializer = CSVDeserializer(accept='text/csv') # 디폴트가 accept가 'text/csv' 이지만 직관적 이유로 기술함.

from src.sm_utils import get_predictor

predictor = get_predictor(local_endpoint_name, local_session, csv_deserializer)



zjbmhkpbn8-algo-1-559cq | [2021-11-10 01:48:21 +0000] [1781] [INFO] Booting worker with pid: 1781


## 로컬 프리딕터에서 추론

In [40]:
result = predictor.predict(payload, initial_args = {"ContentType": "text/csv"})
result


zjbmhkpbn8-algo-1-559cq | [2021-11-10 01:48:21 +0000] [1912] [INFO] Booting worker with pid: 1912
zjbmhkpbn8-algo-1-559cq | [2021-11-10 01:48:21 +0000] [1909] [INFO] Booting worker with pid: 1909
zjbmhkpbn8-algo-1-559cq | 2021-11-10 01:48:21,392 INFO - sagemaker-containers - No GPUs detected (normal if no gpus installed)
zjbmhkpbn8-algo-1-559cq | [2021-11-10 01:48:21 +0000] [1977] [INFO] Booting worker with pid: 1977
zjbmhkpbn8-algo-1-559cq | [2021-11-10 01:48:21 +0000] [2104] [INFO] Booting worker with pid: 2104
zjbmhkpbn8-algo-1-559cq | [2021-11-10 01:48:21 +0000] [2105] [INFO] Booting worker with pid: 2105
zjbmhkpbn8-algo-1-559cq | [2021-11-10 01:48:21 +0000] [2234] [INFO] Booting worker with pid: 2234
zjbmhkpbn8-algo-1-559cq | [2021-11-10 01:48:21 +0000] [2362] [INFO] Booting worker with pid: 2362
zjbmhkpbn8-algo-1-559cq | [2021-11-10 01:48:21 +0000] [2363] [INFO] Booting worker with pid: 2363
zjbmhkpbn8-algo-1-559cq | [2021-11-10 01:48:21 +0000] [2364] [INFO] Booting worker with p

[['0.8195698233560474', '0.18043017664395256'],
 ['0.8195698233560474', '0.18043017664395256'],
 ['0.8195698233560474', '0.18043017664395256'],
 ['0.8186971275661173', '0.18130287243388263'],
 ['0.8195698233560474', '0.18043017664395256']]

zjbmhkpbn8-algo-1-559cq | 172.18.0.1 - - [10/Nov/2021:01:48:22 +0000] "POST /invocations HTTP/1.1" 200 195 "-" "python-urllib3/1.26.7"


In [41]:
dummy_endpont.delete_endpoint()

zjbmhkpbn8-algo-1-559cq | [2021-11-10 01:48:22 +0000] [3018] [INFO] Booting worker with pid: 3018
Gracefully stopping... (press Ctrl+C again to force)


# 8. 호스트 엔드포인트에서 추론 

In [42]:
endpoint_name = 'host-endpoint-' + str(suffix)
print("endpoint_name: ", endpoint_name)


endpoint_name:  host-endpoint-68739


In [43]:
%%time 

catboost_endpont = sklearn.deploy(
    initial_instance_count=1, 
    instance_type="ml.c5.xlarge",
    endpoint_name = endpoint_name    
)

-----!CPU times: user 112 ms, sys: 4.2 ms, total: 116 ms
Wall time: 2min 32s


## 호스트 프리딕터 생성

In [44]:
host_predictor = get_predictor(endpoint_name, sess, csv_deserializer)

In [45]:
num_inference = 1000
full_payload = get_payload(test_df[0:num_inference], label_col = 'EVENT_LABEL', verbose=False)
#dp(full_payload)

In [46]:
result = host_predictor.predict(full_payload, initial_args = {"ContentType": "text/csv"})
result[0:10]    

[['0.8195698233560474', '0.18043017664395256'],
 ['0.8195698233560474', '0.18043017664395256'],
 ['0.8195698233560474', '0.18043017664395256'],
 ['0.8186971275661173', '0.18130287243388263'],
 ['0.8195698233560474', '0.18043017664395256'],
 ['0.8195698233560474', '0.18043017664395256'],
 ['0.8195698233560474', '0.18043017664395256'],
 ['0.8195698233560474', '0.18043017664395256'],
 ['0.8195698233560474', '0.18043017664395256'],
 ['0.8195698233560474', '0.18043017664395256']]

# 9. 호스트 앤드포인트 제거

In [47]:
# host_predictor.delete_endpoint()